# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770409418677                   -0.52    9.0         
  2   -2.771690016631       -2.89       -1.32    1.0    194ms
  3   -2.771714447999       -4.61       -2.48    1.0    148ms
  4   -2.771714647618       -6.70       -3.20    1.0    169ms
  5   -2.771714711452       -7.19       -3.74    2.0    205ms
  6   -2.771714715135       -8.43       -4.80    1.0    155ms
  7   -2.771714715247       -9.95       -5.24    2.0    187ms
  8   -2.771714715249      -11.72       -5.91    1.0    161ms
  9   -2.771714715250      -12.36       -6.65    2.0    212ms
 10   -2.771714715250      -15.05       -7.32    1.0    173ms
 11   -2.771714715250   +  -14.45       -8.32    1.0    186ms


-0.00013457486662865517

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770471450219                   -0.53    8.0         
  2   -2.771769907532       -2.89       -1.31    1.0    149ms
  3   -2.771801158827       -4.51       -2.59    1.0    189ms
  4   -2.771802068191       -6.04       -3.80    2.0    173ms
  5   -2.771802074040       -8.23       -4.21    2.0    235ms
  6   -2.771802074469       -9.37       -5.52    1.0    158ms
  7   -2.771802074476      -11.17       -6.28    2.0    179ms
  8   -2.771802074476      -13.61       -6.87    1.0    194ms
  9   -2.771802074476      -14.01       -7.43    2.0    193ms
 10   -2.771802074476      -14.10       -7.30    1.0    189ms
 11   -2.771802074476   +  -14.24       -8.05    1.0    175ms


0.01761222219520954

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457486662865517
Displaced dipole:  0.01761222219520954
Polarizability :   1.7746797061838195


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920857341e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551825229e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852772874872e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694603045614e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088787578917e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.284186556721e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 1.022768485721e-12
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 1.022768485721e-12
[ Info: GMRES linsolve in iter 2; step 1: normres = 7.359190077272e-11
[ Info: GMRES linsolve in iter 2; step 2: normres = 1.282988071788e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 2.594233639335e-13
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 2.594233639335e-13
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 2.594403842525164e-13
└ *  number of operat

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.